In [1]:
import csv
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_csv("Online Retail.csv")
#df.loc[df['Quantity'].values < 0]=0 
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

In [3]:
basket = (df[df['Country'] =="United Kingdom"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [4]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [26]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)  #create the rule

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [87]:
temp = rules[ (rules['lift'] >= 5) &
       (rules['confidence'] >= 0.6) ]

In [88]:
temp['antecedants'] = [list(x) for x in temp['antecedants']]    #to transform frozenset to normalset,and to change the string to Conform the format
temp['antecedants'] = temp['antecedants'].astype(str)
temp['antecedants'] = temp['antecedants'].str.replace("\[",'')  #裡面的資料長得像這樣 ['data']，為了要比對submit中的檔案所以將頭尾去掉
temp['antecedants'] = temp['antecedants'].str.replace("\]",'')
temp['antecedants'] = temp['antecedants'].str.replace("\'",'')
temp['consequents'] = [list(x) for x in temp['consequents']]
temp['consequents'] = temp['consequents'].astype(str)
temp['consequents'] = temp['consequents'].str.replace("\[",'')
temp['consequents'] = temp['consequents'].str.replace("\]",'')
temp['consequents'] = temp['consequents'].str.replace("\'",'')

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

In [89]:
def check_label(a,b):                     #判斷output的內容是否符合在rule中confidence>=0.5
    for i,j in zip(temp['antecedants'],temp['consequents']):
        if(i==a and j==b):
            return 1
    return 0

In [90]:
output = pd.read_csv("submit.csv")
answer = []
for i in range(len(output)):
    answer.append([i,check_label(output['Association Rule antecedants'][i],output['Association Rule consequents'][i])])

In [91]:
label = ['index','label']
x = pd.DataFrame.from_records(answer, columns=label)
x.to_csv ("result.csv" ,index=False, encoding = "utf-8")

In [92]:
x1 = pd.read_csv("result.csv")
x2 = pd.read_csv("x2.csv")
t = []
for i,j in zip(x1['label'],x2['label']):
        if(i!=j):
            t.append(1)
print(len(t))

503
